In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_pl = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/t20_pl_g.csv')

In [2]:
df_pl = df_pl.sort_values(by=['start_date','match_id','innings','ball']).reset_index(drop=True)

batter_list = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/player_list.xlsx',
                           sheet_name='season_01')
batters = batter_list['Func_Name'].unique().tolist()

df_pl = df_pl[df_pl['striker'].isin(batters)].reset_index(drop=True)

player_list = df_pl.bowler.unique()

players = player_list.tolist()


play_style_df = pd.read_csv("/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/play_style.csv")


## spin-pace stats

In [6]:
df_pl_mod = df_pl.merge(play_style_df, on='bowler',
                       how='left')


spin_bat = df_pl_mod[df_pl_mod['bowl_style_parent']=='Spin'].groupby('striker').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

spin_bat['SR'] = 100*spin_bat['runs']/spin_bat['balls']
spin_bat['Bat_avg'] = spin_bat['runs']/spin_bat['outs']
spin_bat['bpb'] = spin_bat['balls']/(spin_bat['fours']+spin_bat['sixes'])
spin_bat['dot_%'] = 100*spin_bat['dots']/spin_bat['balls']

pace_bat = df_pl_mod[df_pl_mod['bowl_style_parent']=='Pace'].groupby('striker').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

pace_bat['SR'] = 100*pace_bat['runs']/pace_bat['balls']
pace_bat['Bat_avg'] = pace_bat['runs']/pace_bat['outs']
pace_bat['bpb'] = pace_bat['balls']/(pace_bat['fours']+pace_bat['sixes'])
pace_bat['dot_%'] = 100*pace_bat['dots']/pace_bat['balls']

pace_bat['pace_index_int'] = pace_bat['SR']+np.where(pace_bat['Bat_avg']<1000, pace_bat['Bat_avg'], \
                                pace_bat['runs'])- 10*np.where(pace_bat['bpb']<1000, pace_bat['bpb'], \
                                        pace_bat['balls'])-pace_bat['dot_%']

pace_bat['pace_idx'] = np.where(
    ((pace_bat['balls'] < 300) & (pace_bat['balls'] >= 50)), 
    pace_bat['pace_index_int'] * ((pace_bat['balls'] / 300) ** 0.5),\
    np.where(pace_bat['balls'] >= 300, 
    pace_bat['pace_index_int'], \
            np.nan)
)


##
spin_bat['spin_index_int'] = spin_bat['SR']+np.where(spin_bat['Bat_avg']<1000, spin_bat['Bat_avg'], \
                                spin_bat['runs'])- 10*np.where(spin_bat['bpb']<1000, spin_bat['bpb'], \
                                        spin_bat['balls'])-spin_bat['dot_%']

spin_bat['spin_idx'] = np.where(
    ((spin_bat['balls'] < 300) & (spin_bat['balls'] >= 50)), 
    spin_bat['spin_index_int'] * ((spin_bat['balls'] / 300) ** 0.5),\
    np.where(spin_bat['balls'] >= 300, 
    spin_bat['spin_index_int'], \
            np.nan)
)

##

k1 = 0.02
k2 = 0.1
c1 = 70
c2 = 97

def sigmoid(x):
    if x<=70:
        value = 0.5 + 1 / (1 + np.exp(-k1 * (x-c1)))
    else:
        value = 1 + 1/ (1 + np.exp(-k2 * (x-c2 )))
        
    return value**0.8

pace_bat['pace_index'] = pace_bat['pace_idx'].apply(sigmoid)
spin_bat['spin_index'] = spin_bat['spin_idx'].apply(sigmoid)

##
pace_bat.drop(['pace_index_int','pace_idx'], axis=1, inplace=True)
spin_bat.drop(['spin_index_int','spin_idx'], axis=1, inplace=True)


In [7]:
#pace_bat[pace_bat['balls']>=50].sort_values('pace_index').reset_index()
spin_bat[spin_bat['balls']>=50].sort_values('spin_index').round(2).reset_index()

,index,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,spin_index
0,93,M Theekshana,36,64,8,0,1,39,56.25,4.50,64.00,60.94,0.57
1,66,JD Unadkat,49,56,3,0,2,22,87.50,16.33,28.00,39.29,0.61
2,121,NT Ellis,68,76,10,2,2,33,89.47,6.80,19.00,43.42,0.63
3,172,Shahbaz Ahmed,294,301,15,13,9,118,97.67,19.60,13.68,39.20,0.64
4,4,A Zampa,41,50,5,3,0,23,82.00,8.20,16.67,46.00,0.65
5,75,KH Pandya,507,520,26,22,22,235,97.50,19.50,11.82,45.19,0.65
6,124,Noor Ahmad,65,77,7,3,3,40,84.42,9.29,12.83,51.95,0.66
7,83,Kuldeep Yadav,42,60,1,3,1,33,70.00,42.00,15.00,55.00,0.66
8,162,SN Thakur,99,107,5,4,4,51,92.52,19.80,13.38,47.66,0.66
9,32,Bhanu Pania,69,85,1,1,4,44,81.18,69.00,17.00,51.76,0.67


In [331]:
spin_bat.sample(4)

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,spin_index
115,N Jagadeesan,430,313,10,38,17,100,137.380192,43.000000,5.690909,31.948882,1.366380
173,Shashank Singh,266,228,11,12,16,91,116.666667,24.181818,8.142857,39.912281,0.757334
33,C Green,453,305,6,30,24,95,148.524590,75.500000,5.648148,31.147541,1.980915
139,R Sai Kishore,7,11,2,0,0,4,63.636364,3.500000,inf,36.363636,NaN


In [336]:
batters = 'Rahul|Simran|VR Iyer|Livingstone|Markram|Bethell|Ferreira|Lomror|Mankad|Arya|Badoni|Tewatia|Duckett|KH Pandya|Ali|Santner'

spin_bat[spin_bat['striker'].str.contains(batters)].sort_values(['SR','Bat_avg'],ascending=False)

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,spin_index
31,BM Duckett,1249,783,31,153,25,198,159.514687,40.290323,4.398876,25.287356,1.966199
100,MM Ali,1721,1166,67,105,117,434,147.598628,25.686567,5.252252,37.221269,1.206548
67,JG Bethell,436,297,10,30,22,89,146.801347,43.600000,5.711538,29.966330,1.641107
37,D Ferreira,424,292,19,23,29,103,145.205479,22.315789,5.615385,35.273973,1.100399
126,P Simran Singh,615,435,15,49,34,170,141.379310,41.000000,5.240964,39.080460,1.351813
97,MJ Santner,678,487,27,49,34,173,139.219713,25.111111,5.867470,35.523614,1.063760
193,VR Iyer,674,503,15,52,26,159,133.996024,44.933333,6.448718,31.610338,1.195161
78,KL Rahul,1985,1496,47,144,70,435,132.687166,42.234043,6.990654,29.077540,1.108487
98,MK Lomror,674,508,20,37,38,174,132.677165,33.700000,6.773333,34.251969,0.971989
7,AK Markram,1264,956,41,70,57,273,132.217573,30.829268,7.527559,28.556485,0.946282


In [337]:
batters = 'Rahul|Simran|VR Iyer|Livingstone|Markram|Bethell|Ferreira|Lomror|Mankad|Arya|Badoni|Tewatia|Duckett|KH Pandya|Ali|Santner'

pace_bat[pace_bat['striker'].str.contains(batters)].sort_values(['SR','Bat_avg'],ascending=False)

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,pace_index
40,D Ferreira,789,481,24,61,44,193,164.033264,32.875000,4.580952,40.124740,1.801771
91,LS Livingstone,3019,1870,96,246,190,767,161.443850,31.447917,4.288991,41.016043,1.768272
139,Priyansh Arya,185,115,4,23,9,55,160.869565,46.250000,3.593750,47.826087,1.112791
146,R Tewatia,1135,719,33,105,54,258,157.858136,34.393939,4.522013,35.883171,1.804534
135,PN Mankad,489,312,14,58,17,111,156.730769,34.928571,4.160000,35.576923,1.851731
107,MM Ali,2393,1655,84,206,132,724,144.592145,28.488095,4.896450,43.746224,1.159353
132,P Simran Singh,1076,746,40,124,52,369,144.235925,26.900000,4.238636,49.463807,1.145365
200,VR Iyer,1537,1087,44,160,63,453,141.398344,34.931818,4.874439,41.674333,1.248084
83,KL Rahul,3953,2845,85,358,181,1163,138.945518,46.505882,5.278293,40.878735,1.372612
0,A Badoni,595,431,18,50,22,173,138.051044,33.055556,5.986111,40.139211,1.069826


In [178]:
ball = 50
sr = 140
avg = 30

spin_bat[(spin_bat['balls']>=ball)&(spin_bat['SR']>=sr)&(spin_bat['Bat_avg']>=avg)]\
                .sort_values(['SR','Bat_avg'],ascending=False).head(50)

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%
24,Ashutosh Sharma,107,61,3,10,6,19,175.409836,35.666667,3.812500,31.147541
13,Abhishek Sharma,1055,606,28,59,80,179,174.092409,37.678571,4.359712,29.537954
148,RM Patidar,881,512,20,46,65,146,172.070312,44.050000,4.612613,28.515625
50,H Klaasen,1550,947,39,75,114,284,163.674762,39.743590,5.010582,29.989440
123,Nithish Kumar Reddy,262,162,4,12,21,59,161.728395,65.500000,4.909091,36.419753
31,BM Duckett,1249,783,31,153,25,198,159.514687,40.290323,4.398876,25.287356
169,Sameer Rizvi,161,101,3,11,12,38,159.405941,53.666667,4.391304,37.623762
198,YBK Jaiswal,815,548,23,76,41,194,148.722628,35.434783,4.683761,35.401460
33,C Green,453,305,6,30,24,95,148.524590,75.500000,5.648148,31.147541
195,Vishnu Vinod,296,200,8,21,14,53,148.000000,37.000000,5.714286,26.500000


In [183]:
spin_bat[spin_bat.striker.str.contains('KL')]

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%
78,KL Rahul,1985,1496,47,144,70,435,132.687166,42.234043,6.990654,29.07754


In [319]:
ball = 50
sr = 140
avg = 30

pace_bat[(pace_bat['balls']>=ball)&(pace_bat['SR']>=sr)&(pace_bat['Bat_avg']>=avg)]\
                .sort_values(['SR','Bat_avg'],ascending=False).head(50)

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,pace_index
177,Sanvir Singh,161,86,4,14,9,32,187.209302,40.250000,3.739130,37.209302,1.180088
7,AD Russell,3773,2048,119,273,319,954,184.228516,31.705882,3.459459,46.582031,1.977594
12,AS Roy,181,100,5,12,10,23,181.000000,36.200000,4.545455,23.000000,1.247465
190,TH David,3316,1929,99,252,204,705,171.902540,33.494949,4.230263,36.547434,1.950487
153,RK Singh,1572,919,39,138,91,353,171.055495,40.307692,4.013100,38.411317,1.972935
164,SB Dubey,171,100,5,16,9,30,171.000000,34.200000,4.000000,30.000000,1.130764
163,SA Yadav,3992,2357,112,418,213,921,169.367840,35.642857,3.735341,39.075095,1.959231
179,Shashank Singh,453,270,15,43,20,79,167.777778,30.200000,4.285714,29.259259,1.903807
133,PD Salt,3631,2187,113,403,158,843,166.026520,32.132743,3.898396,38.545953,1.913957
191,TM Head,1788,1088,50,215,74,431,164.338235,35.760000,3.764706,39.613971,1.929806


In [185]:
pace_bat[pace_bat.striker.str.contains('KL')]

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%
83,KL Rahul,3953,2845,85,358,181,1163,138.945518,46.505882,5.278293,40.878735


In [322]:
pace_bat_trimmed = pace_bat[~pace_bat.pace_index.isna()].reset_index(drop=True)
spin_bat_trimmed = spin_bat[~spin_bat.spin_index.isna()].reset_index(drop=True)

In [324]:
pace_bat_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/pace_bat.csv',
                           index=None)
spin_bat_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/spin_bat.csv',
                           index=None)